<a href="https://colab.research.google.com/github/pokjay/heb-squad/blob/main/preprocess_translate_squad_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pandas as pd

## Convert SQuAD JSON file to Pandas DataFrame

In [2]:
!git clone https://github.com/pokjay/heb-squad

Cloning into 'heb-squad'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 47 (delta 11), reused 33 (delta 5), pack-reused 0
Unpacking objects: 100% (47/47), done.
Checking out files: 100% (17/17), done.


In [3]:
with open('/content/heb-squad/data/original/train-v2.0.json') as f_handle:
    dataset = json.load(f_handle)

In [4]:
def handle_answer(article, context, qa, answer_field):

  rows = []

  for answer in qa[answer_field]:

    q = qa['question']
    qid = qa['id']
    is_impossible = qa['is_impossible']
    answer_end = answer['answer_start'] + len(answer['text'])
                
    context_marked = context[:answer['answer_start']] + \
                    '[0123] ' + \
                    context[answer['answer_start'] : answer_end] + \
                    ' [4567]' \
                    + context[answer_end:]
                
    row = {'article': article['title'],
           'id': qid,
           'context': context_marked,
           'question': q,
           'answer': answer['text'],
           'answer_start': answer['answer_start'],
           'answer_end': answer_end,
           'is_impossible': int(is_impossible)}

    rows.append(row)
  
  return rows

In [5]:
rows = []

for article in dataset['data']:
  for p in article['paragraphs']:
    # Some lines have \n, we remove them as they screw up translation
    context = p['context'].replace('\n', '')
    
    for qa in p['qas']:
      if 'answers' in qa:
        rows.extend(handle_answer(article, context, qa, 'answers'))
      if 'plausible_answers' in qa:
        rows.extend(handle_answer(article, context, qa, 'plausible_answers'))
      


In [6]:
df = pd.DataFrame(rows).fillna("")
df.index = df.index.rename('idx')
df.shape

(130319, 8)

In [7]:
df.head()

,article,id,context,question,answer,answer_start,answer_end,is_impossible
idx,,,,,,,,
0,Beyoncé,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,269,286,0
1,Beyoncé,56be85543aeaaa14008c9065,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,207,226,0
2,Beyoncé,56be85543aeaaa14008c9066,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,526,530,0
3,Beyoncé,56bf6b0f3aeaaa14008c9601,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",166,180,0
4,Beyoncé,56bf6b0f3aeaaa14008c9602,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,276,286,0


In [8]:
import numpy as np

idxs = np.round(np.array([df.shape[0], df.shape[0], df.shape[0], df.shape[0], df.shape[0]]) * np.array([0, 0.25, 0.5, 0.75, 1]))
idxs

array([     0.,  32580.,  65160.,  97739., 130319.])

In [9]:
for i in range(len(idxs) - 1):
  partial_df = df.loc[idxs[i] : idxs[i+1]-1]
  # partial_df.drop(columns=['article']).to_excel(f'/content/heb-squad/data/intermediate/train-v2.0-{i}.xlsx', index=True)


## 1. Translate XLSX manually using Google Translate Document translator
## 2. Run code below processing the translated output

In [10]:
translated_file_format = '/content/heb-squad/data/intermediate/heb-train-v2.0-{}.tsv'#'/Users/akeidar/temp/squad_train_{}_translated.tsv'

In [103]:
dfs = []

for i in range(4):
    dfs.append(pd.read_csv(translated_file_format.format(i), sep='\t', quoting=3, index_col=0))#.dropna(subset=['context_marked'], how='any'))

df2 = pd.concat(dfs)

In [104]:
df2['id'] = df2.id.str.strip()
df2['context'] = df2.context.str.strip()
df2['question'] = df2.question.str.strip()
df2['answer'] = df2.answer.str.strip().astype('str')

Take a quick look at what we got

In [105]:
df2.shape

(130319, 7)

In [106]:
df2.head(3)

,id,context,question,answer,answer_start,answer_end,is_impossible
idx,,,,,,,
0,56be85543aeaaa14008c9063,ביונסה ג'יזל נואלס-קרטר (/ biːˈjɒnseɪ/ bee-YON...,מתי ביונסה החלה להיות פופולרית?,בסוף שנות התשעים,269,286,0
1,56be85543aeaaa14008c9065,ביונסה ג'יזל נואלס-קרטר (/ biːˈjɒnseɪ/ bee-YON...,באילו תחומים ביונסה התחרה כשהתבגר?,שירה וריקודים,207,226,0
2,56be85543aeaaa14008c9066,ביונסה ג'יזל נואלס-קרטר (/ biːˈjɒnseɪ/ bee-YON...,מתי עזבה ביונסה את Destiny's Child והפכה לזמרת...,2003,526,530,0


### Validate IDs are correct, fix if needed

Check how many diffs we have

In [107]:
pd.Series(df.id.values == df2.id.values).value_counts()

True     129721
False       598
dtype: int64

Print problematic IDs

In [108]:
df2.loc[~(df.id.values == df2.id.values), 'id']

idx
518       56bec4de3aeaaa14008c93 להיות
904          56cbe2fd6d243a140015 ערוך
937           56cbeb396d243a140015 עדב
938           56cbeb396d243a140015 אדי
1028                                38
                      ...             
129776      57359bbcdc94161900571 איאה
129778         57359bbcdc94161900571ec
129779      57359bbcdc94161900571 הניח
129980      עמלת 5735c3d0dc94161900571
129986       5735c421dc94161900571 קפה
Name: id, Length: 598, dtype: object

Print original IDs

In [109]:
df.loc[~(df.id.values == df2.id.values), 'id']

idx
518       56bec4de3aeaaa14008c93be
904       56cbe2fd6d243a140015edce
937       56cbeb396d243a140015edeb
938       56cbeb396d243a140015edec
1028      56cbfbdd6d243a140015ee38
                    ...           
129776    57359bbcdc94161900571eea
129778    57359bbcdc94161900571eec
129779    57359bbcdc94161900571eed
129980    5735c3d0dc94161900571fee
129986    5735c421dc94161900571ffe
Name: id, Length: 598, dtype: object

Replace problematic ids with original ids

In [110]:
df2.loc[~(df.id.values == df2.id.values), 'id'] = df.loc[~(df.id.values == df2.id.values), 'id'].to_list()

In [111]:
pd.Series(df.id.values == df2.id.values).value_counts()

True    130319
dtype: int64

### Add name of article for creating JSON later

In [112]:
df2['article'] = df.article.to_list()
df2.article.value_counts() == df.article.value_counts()

Queen_Victoria                                    True
New_York_City                                     True
American_Idol                                     True
Beyoncé                                           True
Frédéric_Chopin                                   True
                                                  ... 
Order_of_the_British_Empire                       True
Letter_case                                       True
Race_and_ethnicity_in_the_United_States_Census    True
Grape                                             True
Pitch_(music)                                     True
Name: article, Length: 442, dtype: bool

Remove markers from context, and then print a sample to see if context looks good

In [113]:
df2['context_marked'] = df2.context

In [114]:
df2['context'] = df2.context_marked.str.replace('[0123] ', '', regex=False)
df2['context'] = df2.context.str.replace(' [4567]', '', regex=False)

In [115]:
df2.iloc[10].context_marked, '', '', df2.iloc[10].context

('ביונסה ג\'יזל נואלס-קרטר (/ biːˈjɒnseɪ/ bee-YON-say) (נולדה ב -4 בספטמבר 1981) היא זמרת, כותבת שירים, מפיקה ושחקנית אמריקאית. היא נולדה וגדלה ביוסטון, טקסס, והופיעה בתחרויות שירה וריקוד בילדותה, ועלתה לתהילה בסוף שנות ה -90 כזמרת להקת הבנות R & B Destiny\'s Child. בניהולו של אביה, מתיו נואלס, הפכה הקבוצה לאחת מקבוצות הנערות הנמכרות ביותר בכל הזמנים. ההפסקה שלהם יצאה לאור אלבום הבכורה של ביונסה, [0123] Dangerously in Love [4567] (2003), שהקים אותה כאמנית סולו ברחבי העולם, זכתה בחמישה פרסי גראמי והציגה את סינגלי בילבורד הוט 100 מספר אחת "Crazy in Love "ו-" בייבי בוי ".',
 '',
 '',
 'ביונסה ג\'יזל נואלס-קרטר (/ biːˈjɒnseɪ/ bee-YON-say) (נולדה ב -4 בספטמבר 1981) היא זמרת, כותבת שירים, מפיקה ושחקנית אמריקאית. היא נולדה וגדלה ביוסטון, טקסס, והופיעה בתחרויות שירה וריקוד בילדותה, ועלתה לתהילה בסוף שנות ה -90 כזמרת להקת הבנות R & B Destiny\'s Child. בניהולו של אביה, מתיו נואלס, הפכה הקבוצה לאחת מקבוצות הנערות הנמכרות ביותר בכל הזמנים. ההפסקה שלהם יצאה לאור אלבום הבכורה של ביונסה, Dangerously 

Calculate the new start and end locations of the answer in the Hebrew context

In [116]:
df2['answer_start_heb'] = df2.context_marked.str.find('[0123] ')
df2['answer_end_heb'] = df2.apply(lambda row: row.context_marked.find(' [4567]', int(row.answer_start_heb)) - len(' [4567]'), axis=1)

Validate how many rows are problematic and will be discarded

In [117]:
print((df2.answer_start_heb >= 0).value_counts())
print()
print((df2.answer_end_heb >= 0).value_counts())

True     114467
False     15852
Name: answer_start_heb, dtype: int64

True     112644
False     17675
Name: answer_end_heb, dtype: int64


In [118]:
for row in df2.sample(30):
  df2.iloc[10].context[int(df2.iloc[10].answer_start_heb):int(df2.iloc[10].answer_end_heb)]

In [119]:
for row in df2.sample(10).itertuples():
  print(row.question)
  print(row.context[int(row.answer_start_heb):int(row.answer_end_heb)])
  print()

באיזה סרט כיכבה ביונסה עם סטיב מרטין?
הפנתר הוורוד

מאיפה מקור הביטוי הצרפתי "ביטומן"?


איזה כת נוצרה בארס מקסימה בפורום בוארום?
הרקולס

מה לא זיהה את הסופות על נפטון?
וויאג'ר 2

יחד עם חומצות שומן, אילו חומצות נמצאות בתמציות העץ?
שרף

לאיזו מפלגה פוליטית משויך בעל הטור של הטיימס, דניאל פינקלשטיין?


מתי התכוון הנסיך לואיס לנחות בדרום אנגליה?
מאי 1216

היכן נמצאת אגורדת?
במרכז אריתריאה

האנגלים כבשו את ניו יורק החל מאיזה תאריך?
1664

אף מדינה אחת לא מורכבת מכמה מהאוכלוסייה לאחר מלחמת העולם השנייה?
30%



#### Try and fix answers ruined in translation
These are rows where the tags surrounding the answer was lost in translation, maeaning we cannot retrieve the correct indices of the answers from the Hebrew translation. We will try to use the translated answer to find the indices in the context.

In [120]:
mis_ans_df = df2[(df2.answer_start_heb >= df2.answer_end_heb) | (df2.answer_start_heb < 0) | (df2.answer_end_heb < 0)]

In [121]:
mis_ans_df.shape

(22895, 11)

In [122]:
new_ans_start = mis_ans_df.apply(lambda row: row.context.find(row.answer), axis=1)
new_ans_end = mis_ans_df.apply(lambda row: -1 if (row.context.find(row.answer) == -1) else (row.context.find(row.answer) + len(row.answer)), axis=1)

new_ans_start_success = new_ans_start[new_ans_start > -1]
new_ans_end_success = new_ans_end[new_ans_end > -1]

df2.loc[new_ans_start_success.index, 'answer_start_heb'] = new_ans_start_success.to_list()
df2.loc[new_ans_end.index, 'answer_end_heb'] = new_ans_end.to_list()

In [123]:
print((df2.answer_start_heb >= 0).value_counts())
print()
print((df2.answer_end_heb >= 0).value_counts())

True     121941
False      8378
Name: answer_start_heb, dtype: int64

True     117895
False     12424
Name: answer_end_heb, dtype: int64


Below we remove rows where the translation deleted the answer markers from the context.

In [125]:
print('We will remove', df2[(df2.answer_start_heb >= df2.answer_end_heb) | (df2.answer_start_heb < 0) | (df2.answer_end_heb < 0)].shape[0], 'rows')

We will remove 12424 rows


In [126]:
df2_clean = df2[(df2.answer_start_heb < df2.answer_end_heb) | (df2.answer_start_heb >= 0) | (df2.answer_end_heb >= 0)]

In [127]:
df2_clean.shape

(121941, 11)

Now we can save the df to file, it is ready!

In [128]:
!mkdir /content/heb-squad/data/final

In [129]:
df2_clean.to_csv('/content/heb-squad/data/final/heb-train-v2.0.csv', index=False)